In [1]:
import os
import pathlib
import numpy as np
import imageio
import glob
import matplotlib.pyplot as plt
import IPython.display
import PIL.Image
import imageio
import cv2

## Input Images Path

In [2]:
# Relative paths to the notebook
images_path = []
images_path += glob.glob("images/*.png")

output_path = 'outputs/'
# If output folder does not exist, create it
p = pathlib.Path(output_path)
if not p.is_dir():
    p.mkdir()

# Execute all dithering functions to all images
# NOTE: THE ENTIRE ACQUISITION TAKES ABOUT 5 MINUTES
execute_acquisitions = True

In [3]:
images_path

['images/baboon.png',
 'images/monalisa.png',
 'images/watch.png',
 'images/peppers.png']

In [4]:
images = {}
for img in images_path:
     images[os.path.split(img)[1]] = cv2.cvtColor(cv2.imread(img,-1), cv2.COLOR_BGR2RGB)

## SVD

In [5]:
def svd(img, k):
    svd = [np.linalg.svd(img[:, :, i]) for i in range(img.shape[2])]
    
    k = min(k, img.shape[0], img.shape[1])
    
    result_red = np.clip(np.dot(svd[0][0][:, 0:k] * svd[0][1][0:k], svd[0][2][0:k, :]), 0, 255)
    result_green = np.clip(np.dot(svd[1][0][:, 0:k] * svd[1][1][0:k], svd[1][2][0:k, :]), 0, 255)
    result_blue = np.clip(np.dot(svd[2][0][:, 0:k] * svd[2][1][0:k], svd[2][2][0:k, :]), 0, 255)
    
    result = np.zeros(list(result_red.shape) + [3], np.uint8)

    result[:, :, 0] = result_red
    result[:, :, 1] = result_green
    result[:, :, 2] = result_blue
    
    return result

In [6]:
def rmse(original, compressed):
    return np.sqrt(np.mean((original - compressed)**2))

## Execution

In [7]:
k_list = [1, 2, 4, 8, 16, 32, 64, 128]

result = {}

for key, img in images.items():
    imageio.imwrite(output_path + 'original_' + key, img)
    for k in k_list:
        result[str(k) + '_' + key] = svd(img, k)
        imageio.imwrite(output_path + str(k) + '_' + key, result[str(k) + '_' + key])

## Compression rate

In [8]:
cwd = os.getcwd()

for key, img in images.items():
    original_size = os.path.getsize(output_path + 'original_' + key)
    for k in k_list:  
        compressed_size = os.path.getsize(cwd + '/' + output_path + str(k) + '_' + key)
        print(key, k, compressed_size/original_size)

baboon.png 1 0.34431043092930785
baboon.png 2 0.4012092823323409
baboon.png 4 0.47112862136757
baboon.png 8 0.543219358631545
baboon.png 16 0.6377067885431087
baboon.png 32 0.7498561883269171
baboon.png 64 0.853296289974904
baboon.png 128 0.9392166530756734
monalisa.png 1 0.4834250886565025
monalisa.png 2 0.5230506906590965
monalisa.png 4 0.6153078716090589
monalisa.png 8 0.7155282657790435
monalisa.png 16 0.8046473239794298
monalisa.png 32 0.9025640095413443
monalisa.png 64 1.01414876152988
monalisa.png 128 1.076385172823494
watch.png 1 0.5258444311220244
watch.png 2 0.5915370553949173
watch.png 4 0.7015403107135334
watch.png 8 0.8600227725003645
watch.png 16 1.0170071985258267
watch.png 32 1.2441400582422164
watch.png 64 1.5026889520967346
watch.png 128 1.6742295132193923
peppers.png 1 0.3452983153207351
peppers.png 2 0.3928983872808553
peppers.png 4 0.45553657682613313
peppers.png 8 0.524039903047279
peppers.png 16 0.6017394231197033
peppers.png 32 0.6847547069075912
peppers.png 64 

## RMSE

In [9]:
for key, img in images.items():
    for k in k_list:
        print(key, k, rmse(img, result[str(k) + '_' + key]))

baboon.png 1 10.226570146876915
baboon.png 2 10.000828899362112
baboon.png 4 9.758676173580477
baboon.png 8 9.524311279067698
baboon.png 16 9.351483091071927
baboon.png 32 9.190073911350972
baboon.png 64 8.838968123197988
baboon.png 128 7.9203299094695065
monalisa.png 1 9.688691752972359
monalisa.png 2 9.12878924561304
monalisa.png 4 8.656841969360825
monalisa.png 8 7.889491074671275
monalisa.png 16 6.8112862815271
monalisa.png 32 5.524896395763387
monalisa.png 64 3.7365338457806136
monalisa.png 128 1.4831255020674456
watch.png 1 9.721954046516814
watch.png 2 9.34749235455955
watch.png 4 8.73050795320399
watch.png 8 8.380149308449814
watch.png 16 7.928081514771129
watch.png 32 7.341575527888374
watch.png 64 6.665827253265539
watch.png 128 4.978212017653905
peppers.png 1 10.303829671241736
peppers.png 2 9.761478090875176
peppers.png 4 9.551221525085497
peppers.png 8 9.057287922895528
peppers.png 16 8.228442917556972
peppers.png 32 7.091875139422863
peppers.png 64 5.626271083057484
peppe